# Women's Age at Childbirth

Exploring data of number of children born by year and age of the mother. The data can be found in the SCB table [Levande födda efter region, moderns ålder och barnets kön](http://www.statistikdatabasen.scb.se/pxweb/sv/ssd/START__BE__BE0101__BE0101H/FoddaK/?rxid=e9488ae4-e81e-410e-b7ce-760a5828ec57). Select "Riket" as region, all unique values for the mother's age, both genders, and all years.

Note that in order to generate a GIF, [ImageMagick](https://www.imagemagick.org/) needs to be installed and accessible from the command-line.

- Date: 2019-02-21
- Source: SCB

## Setup

In [1]:
import pathlib
import shutil
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from src.plot import ds_plot

%config InlineBackend.figure_format = 'retina'
plt.ioff()

# Custom chart style
if 'datastory' in plt.style.available:
    plt.style.use('datastory')

In [2]:
df = pd.read_csv('../data/raw/childbirth.csv', encoding='latin1', skiprows=1)

## Transform

In [3]:
df = df.drop('region', axis=1)

# Add boys and girls
m = df[df['kön'] == 'män'].drop('kön', axis=1).set_index('moderns ålder')
k = df[df['kön'] == 'kvinnor'].drop('kön', axis=1).set_index('moderns ålder')
df = m + k

df = df.drop('uppgift saknas')
df = df.reset_index()

In [4]:
df['moderns ålder'] = (df['moderns ålder']
                           .str.replace(' år', '')
                           .str.replace('-', '')
                           .str.replace('+', '')
                           .astype(int))

In [5]:
# Calculate percentages
mothers = df['moderns ålder'].copy()
df = df.div(df.sum(), axis=1)
df['moderns ålder'] = mothers

In [6]:
df = (df.melt(id_vars='moderns ålder',
             value_vars=[str(x) for x in range(1968, 2019)],
             var_name='year')
    .rename(columns={'moderns ålder': 'mother'})
    .assign(year=lambda x: x.year.astype(int)))

## Plot

In [7]:
w = 12
h = w * 0.597
figsize = (w, h)

fig, ax = ds_plot(figsize=figsize)

# Create directory for GIF frames
TEMP_IMG_DIR = '.tmp'
pathlib.Path(TEMP_IMG_DIR).mkdir(exist_ok=True)

# Colors
c_old = '#2bbaff'
c_new = '#f85242'

textargs = {'size': 20, 'weight': 'bold',
            'ha': 'left', 'backgroundcolor': '#ffffff'}

plt.tight_layout(rect=(0.01, 0.02, 1, 1))

for i in range(1968, 2019):
    years = [i, 2018]

    df[df.year == years[0]].plot.area(x='mother', y='value',
                                      color=c_old, alpha=0.7,
                                      legend=False, ax=ax, lw=0)
    df[df.year == years[1]].plot.area(x='mother', y='value',
                                      color=c_new, alpha=0.7,
                                      legend=False, ax=ax, lw=0)

    ax.text(17, 0.07, years[0], color=c_old, **textargs)
    ax.text(36, 0.07, years[1], color=c_new, **textargs)

    # Ticks and labels
    vals = [0, 0.02, .04, .06, .08, .1]
    ax.set_yticks(vals)
    ax.set_yticklabels(['{:,.0%}'.format(x) for x in vals])
    ax.set_ylim(0, 0.1)
    ax.set_xlabel('Moderns ålder', fontdict={'size': 14, 'weight': '500', 'color': '#606062'})
    ax.text(14, 0.1, 'Andel födda', color='#606062', fontdict={'size': 14, 'weight': 500},
            ha='left', va='center')
    
    # Save image
    fig.savefig(f'{TEMP_IMG_DIR}/{i}.png', dpi=150)
    
    # Resize image
    !convert {TEMP_IMG_DIR}/{i}.png -resize 65% -quality 100 {TEMP_IMG_DIR}/{i}.png
    
    # Paste image onto template image
    !convert ../assets/chart-templates/childbirth-template.png \
        {TEMP_IMG_DIR}/{i}.png \
        -gravity northwest \
        -geometry +20+120 \
        -composite {TEMP_IMG_DIR}/{i}.png

    ax.clear()

# Create GIF and remove intermediate frames
!convert -loop 0 -delay 100 {TEMP_IMG_DIR}/*.png ../charts/2019-02-21-childbirth.gif
shutil.rmtree(TEMP_IMG_DIR)

# Display GIF
plt.close();
display(Markdown("![childbirth-gif](../charts/2019-02-21-childbirth.gif)"))

![childbirth-gif](../charts/2019-02-21-childbirth.gif)